# Teste de Classificadores com ajuste de peso e modelos de arvore com up/downsampling 

O objetivo é testar os modelos XGBoost e adaBoost sem sampling e a floresta aleatoria e arvore de decisão com sampling, analisar qual deles tem a melhor metrica de recall para o banco de dados com todas as 4 tabelas e qual tem o melhor ponto de cotovelo na curva recall-precision.

## Preparar os dados e criar classe de processamento 

### Importando bibliotecas e dados

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import islice

from tqdm.auto import tqdm
import spacy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords

import xgboost as xgb
from xgboost import plot_tree

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from enelvo.normaliser import Normaliser

In [12]:
df_denuncia_crime=pd.read_csv('/Users/mhctds/Cidade_Social/base de dados/base_apps_denuncia_crime.csv')
df_base_rocinha=pd.read_csv('/Users/mhctds/Cidade_Social/base de dados/base_rocinha_df.csv')
df_protestos_2013=pd.read_csv('/Users/mhctds/Cidade_Social/base de dados/protestos_2013_df.csv')
df_protestos_PMES=pd.read_csv('/Users/mhctds/Cidade_Social/base de dados/protestos_PMES.csv')

### Criar classe de processamento

In [13]:
class df_processing:
    # Junta os dataframes dentro do vetor de dataframes
    def append_data(self,df_vector):
        df_final=df_vector[0]
        df_final=df_final[['text','Total(SUM)','Classe de Violência']]
        for df in islice(df_vector, 1, None) :
            df=df[['text','Total(SUM)','Classe de Violência']]
            df_final=pd.concat([df_final]+[df])
        df_final.reset_index()
        df_final['text']=df_final['text'].astype('str')
        return df_final
    
    #Normalização usando spacy
    def text_normalizer_spacy(self,corpus):
        nlp = spacy.load('pt_core_news_sm', disable=['parser', 'ner'])
        lemm=[]
        for text in tqdm(corpus):
            doc = nlp(text)
            #tokens = [token.lemma_ for token in doc if not token.is_stop]
            tokens = [token.lemma_ for token in doc]
            text= ' '.join(tokens)
            
            lemm.append(text)
        return lemm
    # Normalização usando enelvo e vetorização usando nltk 
    def text_preprocessing_nltk(self,corpus):
        stop_words=list(nltk_stopwords.words('portuguese'))
        norm = Normaliser(tokenizer='readable',sanitize=True)
        lemm=[]
        for texts in corpus:
            lemm.append(norm.normalise(texts))
        print(lemm)
        vect=TfidfVectorizer(stop_words=stop_words)
        processed=vect.fit_transform(lemm)
        return processed
    #Vetorização usando nltk
    def text_preprocessing_nltk_no_norm(self,corpus):
        stop_words=list(nltk_stopwords.words('portuguese'))
        vect=TfidfVectorizer(stop_words=stop_words)
        processed=vect.fit_transform(corpus)
        return processed
    #Mudar target para valor numerico
    def numerical_target(target):
        target.replace('Not Violence',0,inplace=True)
        target.replace('Low',1,inplace=True)
        target.replace('Medium',2,inplace=True)
        target.replace('High',3,inplace=True)
        target.replace('VeryHight',4,inplace=True)
        return target
    #fraction é a fração que vai sobrar do original, deve ser colocado um valor entre 0 e 1
    # Se usado 0.3 por exemplo, perderemos 60% dos registros daquele target, sobrando 30 porcento
    def downsample(self,features, target, fraction,value):
        features_true = features[target == value]
        features_false = features[target != value]
        target_true = target[target == value]
        target_false = target[target != value]

        features_downsampled = pd.concat(
            [features_true.sample(frac=fraction, random_state=12345)]
            + [features_false]
        )
        target_downsampled = pd.concat(
            [target_true.sample(frac=fraction, random_state=12345)]
            + [target_false]
        )

        return features_downsampled, target_downsampled
    # repeat é o numero de vezes que aquele target sera clonado, deve ser um int maior que 1
    def upsample(self,features, target, repeat,value):
        features_true = features[target == value]
        features_false = features[target != value]
        target_true = target[target == value]
        target_false = target[target != value]

        features_upsampled = pd.concat([features_false] + [features_true] * repeat)
        target_upsampled = pd.concat([target_false] + [target_true] * repeat)

        return features_upsampled, target_upsampled

### Unir dataframes e separar em features e target

In [14]:
df_vector=[df_base_rocinha,df_denuncia_crime,df_protestos_2013,df_protestos_PMES]
df_final=df_processing().append_data(df_vector)
df_final.info()
print(df_final.head(5))

<class 'pandas.core.frame.DataFrame'>
Index: 2038 entries, 0 to 503
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 2038 non-null   object 
 1   Total(SUM)           2037 non-null   float64
 2   Classe de Violência  2038 non-null   object 
dtypes: float64(1), object(2)
memory usage: 63.7+ KB
                                                text  Total(SUM)  \
0  RT @tvjornalsbt: "Morte" denuncia a precarieda...         8.0   
1  RT @tvjornalsbt: "Morte" denuncia a precarieda...         8.0   
2  "Morte" denuncia a precariedade da BR-101, na ...         8.0   
3  Terremoto de 7.1 no México. Onde vivo não pass...         7.0   
4  RT @vinigrilo1: Moradores com medo e assustado...         8.0   

  Classe de Violência  
0                High  
1                High  
2                High  
3                High  
4                High  


In [15]:
df_final=df_final.drop_duplicates().reset_index()
features=df_processing().text_preprocessing_nltk(df_final['text'])
target=df_final['Classe de Violência']
print(features)
train_data, test_data, train_target, test_target = train_test_split(features, target, test_size=0.3, random_state=12345,shuffle=True)

['retweeet tvjornalsbt morte denuncia a precariedade da brasileiro 101 na zona sul do recife httpstcorhgy3oujgo httpstcoke84v6mpn1', 'morte denuncia a precariedade da brasileiro 101 na zona sul do recife httpstcorhgy3oujgo httpstcoke84v6mpn1', 'terremoto de 71 no méxico onde vivo não passou quase nada mas na zona sul caíram muitos edifícios orem e ajude o povo mexicano', 'retweeet vinigrilo1 moradores com medo e assustados o fruto do sustento ao tráfico é esse rocinha httpstcofdpzzlhnlg', 'retweeet gloriafperez não dá pra entender o silencio em torno do que está acontecendo na rocinha bandidos barbarizando e ninguém diz nada', 'não dá pra entender o silencio em torno do que está acontecendo na rocinha bandidos barbarizando e ninguém diz nada', 'retweeet vinigrilo1 moradores com medo e assustados o fruto do sustento ao tráfico é esse rocinha httpstcofdpzzlhnlg', 'retweeet felipeoliverrj enquanto isso na rocinha maior favela da américa latina o caos impera não foi lá que entraram até com

## Testando Modelos de Classificação

### XGBoost

In [7]:
# Consertar kernel e dependencias 
"""
best_recall=0
recall_list_XGBoost=[]
precision_list_XGBoost=[]
xgb_train_target=df_processing.numerical_target(train_target)
xgb_test_target=df_processing.numerical_target(test_target)
#Talvez não tenha parametro random_state
#Tem parametro learning rate, talvez vale a pena testar
for estimators in tqdm(range(1,100)):
    for depth in range(1,15):
        model = xgb.XGBClassifier(n_estimators=estimators,max_depth=depth, random_state=12345)
        model.fit(train_data,xgb_train_target)
        prediction=model.predict(test_data)
        recall=recall_score(xgb_test_target,prediction)
        precision=precision_score(xgb_test_target,prediction)
        recall_list_XGBoost.append(recall)
        precision_list_XGBoost.append(precision)
        if best_recall<recall:
            best_depth=depth
            best_recall=recall
            best_estimators=estimators
            best_acc=accuracy_score(test_target,prediction)
            best_f1=f1_score(test_target,prediction)
            best_precision=precision
print('best depth:',best_depth)
print('best number of estimators:',best_estimators)
print('best recall:',best_recall)
print('best accuracy:',best_acc)
print('best f1:',best_f1)
print('best precision:',best_precision) 
"""

  0%|          | 0/99 [00:00<?, ?it/s]

: 

### AdaBoost

In [16]:
best_recall=0
recall_list_adaboost=[]
precision_list_adaboost=[]
#Talvez não tenha parametro  ou random_state
#Talvez tenha parametro learning rate, talvez vale a pena testar
for estimators in tqdm(range(1,100)):
    for depth in range(1,10):
        model = AdaBoostClassifier(n_estimators=estimators, random_state=12345)
        model.fit(train_data,train_target)
        prediction=model.predict(test_data)
        recall=recall_score(test_target,prediction,average='weighted')
        precision=precision_score(test_target,prediction,average='weighted',zero_division=0)
        recall_list_adaboost.append(recall)
        precision_list_adaboost.append(precision)
        if best_recall<recall:
            best_depth=depth
            best_recall=recall
            best_estimators=estimators
            best_acc=accuracy_score(test_target,prediction)
            best_f1=f1_score(test_target,prediction,average='weighted')
            best_precision=precision
print('best depth:',best_depth)
print('best number of estimators:',best_estimators)
print('best recall:',best_recall)
print('best accuracy:',best_acc)
print('best f1:',best_f1)
print('best precision:',best_precision)

100%|██████████| 99/99 [05:41<00:00,  3.45s/it]

best depth: 1
best number of estimators: 1
best recall: 0.7348484848484849
best accuracy: 0.7348484848484849
best f1: 0.670388091440723
best precision: 0.691673670796844


### Floresta Aleatoria sem Sampling

In [ ]:
best_recall=0
recall_list_florest=[]
precision_list_florest=[]
for size in tqdm(range(1,50)):
    for depth in range(1,30):
        model=RandomForestClassifier(random_state=123456789,max_depth=depth,n_estimators=size)
        model.fit(train_data,train_target)
        prediction=model.predict(test_data)
        recall=recall_score(test_target,prediction,average='weighted')
        precision=precision_score(test_target,prediction,average='weighted',zero_division=0)
        recall_list_florest.append(recall)
        precision_list_florest.append(precision)
        if best_recall<recall:
            best_depth=depth
            best_recall=recall
            best_size=size
            best_acc=accuracy_score(test_target,prediction)
            best_f1=f1_score(test_target,prediction,average='weighted')
            best_precision=precision
print('best depth:',best_depth)
print('best size:',best_size)
print('best recall:',best_recall)
print('best accuracy:',best_acc)
print('best f1:',best_f1)
print('best precision:',best_precision)

### Arvore de decisão sem Sampling

In [ ]:
best_recall=0
recall_list_tree=[]
precision_list_tree=[]
for depth in tqdm(range(1,100)):
    model=DecisionTreeClassifier(random_state=123456789,max_depth=depth)
    model.fit(train_data,train_target)
    prediction=model.predict(test_data)
    recall=recall_score(test_target,prediction,average='weighted')
    precision=precision_score(test_target,prediction,average='weighted',zero_division=0)
    recall_list_tree.append(recall)
    precision_list_tree.append(precision)
    if best_recall<recall:
        best_depth=depth
        best_recall=recall
        best_acc=accuracy_score(test_target,prediction)
        best_f1=f1_score(test_target,prediction,average='weighted')
        best_precision=precision
print('best depth:',best_depth)
print('best recall:',best_recall)
print('best accuracy:',best_acc)
print('best f1:',best_f1)
print('best precision:',best_precision)

#### Sampling os dados

In [19]:
features_sampled,target_sampled=df_processing().upsample(df_final['text'], df_final['Classe de Violência'], 8,'Low')
features_sampled,target_sampled=df_processing().upsample(features_sampled,target_sampled, 5,'Medium')
features_sampled,target_sampled=df_processing().upsample(features_sampled,target_sampled, 5,'High')
features_sampled=df_processing().text_preprocessing_nltk(features_sampled)
train_data, test_data, train_target, test_target = train_test_split(features, target, test_size=0.3, random_state=12345,shuffle=True)

['rio de janeiro urgenteamorecos preciso de 03 carros para gravar amanhã as 1 3h na zona sul por favor chamar whatsappp httpstcomul92yjzzg', 'trânsitorj httpstcob2msx5abn0', 'paz na rocinha', 'eu moro no morro e ela na zona sul', 'rocinha httpstcohgnk1idoai', 'retweeet viinloserville poars zona sul esse gatinho extremamente dócil foi abandonado perto de casa e está procurando por um humano pra dar', 'não moro na rocinha mas sigo faltando a escola', 'esse é o morro dá rocinha a verdade prevalece ����', 'retweeet gloriafperez rocinha httpstcohgnk1idoai', 'zona sul é um universo e o filho está pagando de loco the gradue tuas laje é triplex no morro os moleque', 'httpstcottgcydai5c', 'retweeet jornaloglobo se você ainda não viu veja o melhor da noite desta quinta terremoto no méxico e a guerra de versões na rocinha tops', 'retweeet vilanormatd2 guerra na rocinha atinge o botafogo agora conhecidos como bonde do nem nem carioca nem brasileiro nem copa do brasil', 'meu boy aka michael5sos est

### Floresta Aleatoria com Sampling

In [17]:
best_recall=0
recall_list_florest=[]
precision_list_florest=[]
for size in tqdm(range(1,50)):
    for depth in range(1,30):
        model=RandomForestClassifier(random_state=123456789,max_depth=depth,n_estimators=size)
        model.fit(train_data,train_target)
        prediction=model.predict(test_data)
        recall=recall_score(test_target,prediction,average='weighted')
        precision=precision_score(test_target,prediction,average='weighted',zero_division=0)
        recall_list_florest.append(recall)
        precision_list_florest.append(precision)
        if best_recall<recall:
            best_depth=depth
            best_recall=recall
            best_size=size
            best_acc=accuracy_score(test_target,prediction)
            best_f1=f1_score(test_target,prediction,average='weighted')
            best_precision=precision
print('best depth:',best_depth)
print('best size:',best_size)
print('best recall:',best_recall)
print('best accuracy:',best_acc)
print('best f1:',best_f1)
print('best precision:',best_precision)

100%|██████████| 49/49 [01:21<00:00,  1.66s/it]

best depth: 26
best size: 14
best recall: 0.821969696969697
best accuracy: 0.821969696969697
best f1: 0.7883088358696742
best precision: 0.7776370756231701


### Arvore de decisão com Sampling

In [18]:
best_recall=0
recall_list_tree=[]
precision_list_tree=[]
for depth in tqdm(range(1,100)):
    model=DecisionTreeClassifier(random_state=123456789,max_depth=depth)
    model.fit(train_data,train_target)
    prediction=model.predict(test_data)
    recall=recall_score(test_target,prediction,average='weighted')
    precision=precision_score(test_target,prediction,average='weighted',zero_division=0)
    recall_list_tree.append(recall)
    precision_list_tree.append(precision)
    if best_recall<recall:
        best_depth=depth
        best_recall=recall
        best_acc=accuracy_score(test_target,prediction)
        best_f1=f1_score(test_target,prediction,average='weighted')
        best_precision=precision
print('best depth:',best_depth)
print('best recall:',best_recall)
print('best accuracy:',best_acc)
print('best f1:',best_f1)
print('best precision:',best_precision)

100%|██████████| 99/99 [00:04<00:00, 22.32it/s]

best depth: 14
best recall: 0.8238636363636364
best accuracy: 0.8238636363636364
best f1: 0.8099456347364743
best precision: 0.8091923620637237


### Curva Recall-Precision dos modelos

# Conclusões

Por enquanto o melhor modelo entre eles, sem contar o XGBoost, parece ser a arvore de decisão.

Parece valer a pena testar a vetorização com por NILC quando testarmos regressão.

Normalização da uma pequena melhora em alguns dos modelos, mas talvez seja por coincidencia (a seed ser boa para aquele novo formato de matriz especifica)